# Scraping sederhana member JKT48 dari website JKT48
## Menampilkan list nama tiap tim, dan jumlah member tiap tim
Notebook ini saya buat untuk saya belajar jupyter notebooks

In [ ]:
import bs4
import requests

## Scraping member yang telah masuk ke dalam tim J K T
Mengambil informasi dari website member JKT48  

In [ ]:
res = requests.get('https://www.jkt48.com/member/list?lang=id')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('div.profilename > a')
# Print isi dari elems
print(elems)

## Hilangkan tag html
Kita akan mengambil isi dari html dengan menggunakan get_text()  
Maka tampil nama nama member yang diambil dari website jkt48, tidak ada tag html seperti pada hasil eksekusi diatas

In [ ]:
for member in elems:
    print(member.get_text(separator=' ').strip())

## Filter member berdasarkan tim
Sekarang agak sedikit tricky. Membagi member berdasarkan tim dan memasukkannya ke dalam list.  
Dapat dilihat bahwa dari hasil eksekusi diatas, nama ditampilkan berurut berdasarkan huruf depan.  
Jadi setiap tim baru maka akan dimulai dengan huruf A, tetapi sebelumnya harus melewati huruf huruf yang bukan huruf A.  
Perhatikan julie dan tasya  
>Sania Julia Montolalu -> Urutan terakhir untuk tim J  
>Anastasya Narwastu Tety Handuran -> Urutan pertama untuk tim KIII  

Berikut ini kode untuk memfilter tim ke dalam tim J

### Filter tim J

In [ ]:
# Buat variabel bantu untuk menentukan apakah huruf pertama A atau tidak
notA = False
teamJ = []
for member in elems:
    # Cek apakah saat looping kedua dst, ketemu huruf A.
    # Adriani Elisabeth -> looping pertama huruf A
    # Amanina Afiqah -> Kedua ketemu huruf A juga jadi tetep lanjut looping -> Looping ini masuk ke X
    # ...
    # Cindy Hapsari Maharani Pujiantoro Putri -> Cindy bukan lagi A, jadikan variabel notA menjadi True
    # Sampai Sania Julia Montolalu looping akan masuk ke X
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        # Append ke dalam list teamJ
        teamJ.append(member.get_text(separator=' ').strip())
    else:
        # Cek apakah looping kembali lagi ke A dan notA True
        # Sehingga notA menjadi patokan apakah ketemu A lagi, jika ketemu maka break
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            # X
            # Lanjut looping jika looping tidak huruf A
            teamJ.append(member.get_text(separator=' ').strip())

In [ ]:
# Print dan hitung total member tim J
print('Member Team J : \n')
for member in teamJ:
    print(member)
print('\n===Total Member Team J : ' + str(len(teamJ)))

### Filter tim KIII
Tekniknya sama seperti tim J, tapi mulai looping nya dari urutan terakhir tim J  
Jadi variabel elems nya dimulai dari panjang (len) dari variabel teamJ dan diakhiri dari panjang elems itu sendiri

In [ ]:
notA = False
teamKIII = []
# Dapat dilihat elems nya dimulai dan diakhiri seperti ini
# Looping akan start dari Anastasya Narwastu Tety Handuran
for member in elems[len(teamJ):len(elems)]:
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        teamKIII.append(member.get_text(separator=' ').strip())
    else:
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            teamKIII.append(member.get_text(separator=' ').strip())

In [ ]:
# Print dan hitung member tim KIII
print('Member Team KIII : \n')
for member in teamKIII:
    print(member)
print('\n===Total Member Team KIII : ' + str(len(teamKIII)))

### Filter tim T
Tekniknya sama seperti tim J dan KIII, tapi mulai looping nya dari urutan terakhir tim KIII  
Jadi variabel elems nya dimulai dari panjang (len) dari variabel teamKIII dan diakhiri dari panjang elems itu sendiri

In [ ]:
notA = False
teamT = []
# Looping dimulai dari Amirah Fatin
for member in elems[len(teamJ) + len(teamKIII):len(elems)]:
    if member.get_text(separator=' ').strip().upper()[0] != 'A':
        notA = True
        teamT.append(member.get_text(separator=' ').strip())
    else:
        if member.get_text(separator=' ').strip().upper()[0] == 'A' and notA == True:
            break
        else:
            teamT.append(member.get_text(separator=' ').strip())

In [ ]:
print('Member Team T : \n')
for member in teamT:
    print(member)
print('\n===Total Member Team T : ' + str(len(teamT)))

## Mencari kennin member
Kennin member adalah member yang masuk ke dalam lebih dari 1 tim  
Pertama tama kita akan mencari kennin 'J K T', 'J K', dan 'J T'  
Kemudian kita akan mencari kennin 'K T'

In [ ]:
kennin = 0
kenninMember = {}
# Mencari kennin member 'J K T', 'J K', 'J T'
for i in range(len(teamJ)):
    # Jika nama member di tim J ada juga di tim K dan tim T, maka dia triple team J K T
    if teamJ[i] in teamKIII and teamJ[i] in teamT:
        kennin += 2
        kenninMember[teamJ[i]] = ['J', 'K', 'T']
    # Jika nama member di tim J ada juga di tim K, maka dia double team J K
    elif teamJ[i] in teamKIII:
        kennin += 1
        kenninMember[teamJ[i]] = ['J', 'K']
    # Jika nama member di tim J ada juga di tim T, maka dia double team J T
    elif teamJ[i] in teamT:
        kennin += 1
        kenninMember[teamJ[i]] = ['J', 'T']

# Mencari kennin member 'K T'
for i in range(len(teamKIII)):
    if teamKIII[i] in teamT:
        # Jika nama member di tim K ini tidak masuk ke dalam variabel kenninMember, artinya dia double team K T
        # Jika ada artinya dia sudah masuk ke dalam triple team J K T
        if teamKIII[i] not in kenninMember:
            kennin += 1
            kenninMember[teamKIII[i]] = ['K', 'T']

In [ ]:
# Hitung dan print kennin member
print('Kennin : ' + str(len(kenninMember)) + ' Member')
for member, team in kenninMember.items():
    print(member, '=>', team)

## Scraping member Academy
Mengambil informasi dari website academy JKT48

In [ ]:
res = requests.get('https://www.jkt48.com/jkt48-academy/member-academy')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('p.name-member')


In [ ]:
# Memasukkan nama nama member ke dalam variabel academyA
academyA = []
for member in elems:
    academyA.append(member.get_text(separator=' ').strip())

In [ ]:
# Hitung dan print member academy
print('Member Academy Class A : ')
for member in academyA:
    print(member)
print('\n===Total Member Academy A : ' + str(len(academyA)))

## Menghitung total member JKT48
Jumlah tim J + tim K + tim T + academy - kennin  
Variabel kennin berfungsi untuk mengurangi jumlah dari semuanya

In [ ]:
# Hitung jumlah total member JKT48
print('\n===Total Member JKT48 : ' +
      str(len(teamJ) + len(teamKIII) + len(teamT) + len(academyA) - kennin))